In [1]:
!which python

/lfs/skampere1/0/kaif/conda/envs/TheoremSense/bin/python


In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

from vllm import LLM, SamplingParams
from dataset import load_datasets, BOXED_ANSWERS_DATASETS, get_boxed_answer
from latex_formater import latex_deformat
from prompt import generate_nshot_prompts, generate_nshot_prompts_from_template

In [42]:
??PeftModel.from_pretrained

In [38]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from pathlib import Path
from peft import PeftModel, PeftConfig

save_dir = Path("~").expanduser() / "checkpoint" / "checkpoint_0"

config = PeftConfig.from_pretrained(save_dir)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, model_id=save_dir)
model_max_length = model.config.max_position_embeddings
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, model_max_length=model_max_length)
device = "cuda" if torch.cuda.is_available() else "cpu"

# # model_name = "kaist-ai/selfee-7b-delta"
# pipe = pipeline('text-generation', model=model_name, tokenizer=model_name, device_map="auto",
#                 torch_dtype=torch.bfloat16, batch_size=8)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [57]:
model.active_peft_config

PromptTuningConfig(peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, base_model_name_or_path='google/gemma-2b-it', revision=None, task_type='CAUSAL_LM', inference_mode=True, num_virtual_tokens=15, token_dim=2048, num_transformer_submodules=1, num_attention_heads=8, num_layers=18, prompt_tuning_init='TEXT', prompt_tuning_init_text='Is this response correct or not? Provide feedback explaining your reasoning first.', tokenizer_name_or_path='google/gemma-2b-it', tokenizer_kwargs=None)

In [49]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_create_repo',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_base_model_class',
 '_get_files_timestamps',
 '_get_name',
 '_is_full_backward_hook',
 '_is_prompt_learning',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_may

In [60]:
??model.base_model.sample

In [41]:
# test the model
inputs = tokenizer(
    "Hello there my name is", return_tensors="pt", truncation=True
)
model.to(device)

with torch.no_grad():
    # inputs_embeds = self.word_embeddings(model_kwargs["input_ids"])
    # prompts = self.get_prompt(batch_size=model_kwargs["input_ids"].shape[0], task_ids=task_ids)
    # prompts = prompts.to(inputs_embeds.dtype)
    # model_kwargs["inputs_embeds"] = torch.cat((prompts, inputs_embeds), dim=1)


    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], do_sample=True,
        temperature=0.8, top_p=0.95, max_new_tokens=256
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

RuntimeError: The size of tensor a (21) must match the size of tensor b (6) at non-singleton dimension 2

In [2]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=2048)
# model_name = "deepseek-ai/deepseek-math-7b-instruct"
model_name = "google/gemma-2b-it"
# model_name = "Qwen/Qwen1.5-7B-Chat-AWQ"
# model_name = "kaist-ai/selfee-13b-delta"
# model_name = "llm-agents/tora-13b-v1.0"
# model_name = "meta-llama/Llama-2-7b-hf"
# dtype = "bfloat16"
dtype = "float16"
llm = LLM(model=model_name, trust_remote_code=True, dtype=dtype)  #tensor_parallel_size=2,

WARNING 03-13 16:29:17 config.py:577] Casting torch.bfloat16 to torch.float16.
INFO 03-13 16:29:17 llm_engine.py:79] Initializing an LLM engine with config: model='google/gemma-2b-it', tokenizer='google/gemma-2b-it', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
INFO 03-13 16:29:20 weight_utils.py:163] Using model weights format ['*.safetensors']
INFO 03-13 16:29:22 llm_engine.py:337] # GPU blocks: 231898, # CPU blocks: 14563
INFO 03-13 16:29:23 model_runner.py:676] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-13 16:29:23 model_runner.py:680] CUDA graphs can take additi

In [ ]:
while True:
    result = llm.generate(["What is 2+2?" * 100], sampling_params)

Processed prompts:   0%|                                                                              | 0/1 [00:00<?, ?it/s]

In [4]:
%%capture
datasets_raw = load_datasets(BOXED_ANSWERS_DATASETS);

In [5]:
import json

# load the name maps
with open('name_maps.json', 'r') as f:
    name_maps = json.load(f)
    DATASET_MAP = name_maps['DATASET_MAP']
    MODEL_MAP = name_maps['MODEL_MAP']

In [6]:
# NSHOT_MAP = {
#     'Algebra': [0, 1, 2],
#     'Counting and Probability': [0, 1, 2],
#     'Geometry': [1, 6, 8],
#     'Intermediate Algebra': [0, 1, 2],
#     'Number Theory': [0, 1, 2],
#     'Prealgebra': [0, 1, 2],
#     'Precalculus': [4, 6, 8],
#     'GSM8K': [0, 1, 2]
# }

# datasets = {DATASET_MAP[dataset['name']]: generate_nshot_prompts(dataset['data']['train'], n=NSHOT_MAP[DATASET_MAP[dataset['name']]]) for dataset in datasets_raw}
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
datasets = {
    DATASET_MAP[dataset['name']]: generate_nshot_prompts_from_template(dataset['data']['train'], n=3)
    for dataset in datasets_raw
}

import numpy as np

np.random.seed(0)
num_samples = 100
datasets_subset = {k: np.random.choice(v, num_samples, replace=False) for k, v in datasets.items()}
# dtype = 'Algebra'
# datasets_subset = {dtype: datasets_subset[dtype]}

In [10]:
print(datasets_subset['Algebra'][0]['question'])

[{'role': 'user', 'content': 'Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).'}, {'role': 'assistant', 'content': 'For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \\Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\\boxed{0}$.'}, {'role': 'user', 'content': 'FINAL ANSWER: '}, {'role': 'assistant', 'content': '0'}, {'role': 'user', 'content': 'A rectangular band formation is a formation with $m$ band members in each of $r$ rows, where $m$ and $r$ are integers. A particular band has less than 100 

In [7]:
# print(prompts['Algebra'][1]['question'])
# prompt the model for the answers for the question
from util import save_output


@save_output
def generate(chat_prompts, model, tokenizer, sampling_params):
    # assume each chat_prompt is the same length
    chat_pattern = chat_prompts[0]
    last_assistant_idx = [i for i, p in enumerate(chat_pattern) if p['role'] == "assistant"][-1]
    # get the current chat prompts
    curr_prompts = [p[:last_assistant_idx + 1] for p in chat_prompts]
    for i in range(last_assistant_idx + 1, len(chat_pattern)):
        # add the next chat_prompts to the current prompt
        curr_prompts = [cp + [p[i]] for cp, p in zip(curr_prompts, chat_prompts)]
        # convert to chat template and generate the model's response
        prompts = [tokenizer.apply_chat_template(p, tokenize=False, add_generation_prompt=True) for p in
                   curr_prompts]
        outputs = model.generate(prompts, sampling_params)
        # add the model's response to the current prompts
        curr_prompts = [
            p + [{"role": "assistant", "content": o.outputs[0].text}]
            for p, o in zip(curr_prompts, outputs)
        ]
    return curr_prompts

In [12]:
# test the generate function
chat_prompts = [
    [
        {"role": "user", "content": "Hello, how are you?"},
        {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
        {"role": "user", "content": "I'd like to show off how chat templating works!"},
        {"role": "user", "content": "Copy the last model response."}
    ],
    [
        {"role": "user", "content": "Hello, how are you?"},
        {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
        {"role": "user", "content": "I'd like to show off how chat templating works!"},
        {"role": "user", "content": "Copy the last model message."}
    ]
]
outputs = generate(chat_prompts, llm, tokenizer, sampling_params)
print(outputs)

Processed prompts: 100%|██████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.89s/it]

[[{'role': 'user', 'content': 'Hello, how are you?'}, {'role': 'assistant', 'content': "I'm doing great. How can I help you today?"}, {'role': 'user', 'content': "I'd like to show off how chat templating works!"}, {'role': 'assistant', 'content': 'Sure! I\'d be happy to help you demonstrate chat templating. Chat templating allows you to create dynamic messages using placeholders that are replaced with actual values at runtime. This can be useful for creating personalized messages, displaying data from a database, or handling specific user inputs. Here\'s a simple example to get you started:\n\n1. **Template Syntax:**\n   Most chat platforms or libraries use a specific syntax for template placeholders. Let\'s take a common example of a simple template for a user\'s name:\n\n   ```\n   Hi, {{user_name}}!\n   ```\n\n2. **Data to Replace Placeholders:**\n   You need to provide the actual data that will replace the placeholders. For instance, if the user\'s name is "Alice", you would pass i

In [14]:
print(outputs[0][-1]['content'])

Sure! Here's a more detailed example using the popular Python library `jinja2` for chat templating:

1. **Install `jinja2`:**
   If you haven't installed it yet, you can do so with pip:
   ```bash
   pip install jinja2
   ```

2. **Import and Configure:**
   First, create a `Templater` object and set up a template with placeholders:

   ```python
   from jinja2 import Template

   # Define your template
   template_string = """
   Welcome to our system, {{user_name}}!
   Your tasks today are:
   - {{task1}}
   - {{#if task2}}
     - {{task2}}
   {{/if}}
   """

   # Create a template from the string
   template = Template(template_string)
   ```

3. **Pass data to the template:**
   Now, pass the user's name and tasks to the `render` method:

   ```python
   user_name = "Alice"
   tasks = ["Buy groceries", "Finish project"]

   # Render the template
   rendered_message = template.render(user_name=user_name, task1=task1, task2=task2)
   ```

4. **Output the rendered message:**
   The `r

In [8]:
from pathlib import Path

save_dir = Path("~/model_outputs").expanduser()
save_dir.mkdir(exist_ok=True)
load = True
predictions = {
    dataset: generate(
        [p['question'] for p in data], llm, tokenizer, sampling_params, load=load, override=True,
        output_path=save_dir / MODEL_MAP["deepseek-ai/deepseek-math-7b-instruct"] / dataset / "predictions2.pkl"
    ) for dataset, data in datasets_subset.items()
}

Loading output from /lfs/skampere1/0/kaif/model_outputs/Deepseek-7b-Instruct/Algebra/predictions2.pkl
Loading output from /lfs/skampere1/0/kaif/model_outputs/Deepseek-7b-Instruct/Counting_and_Probability/predictions2.pkl
Loading output from /lfs/skampere1/0/kaif/model_outputs/Deepseek-7b-Instruct/Geometry/predictions2.pkl
Loading output from /lfs/skampere1/0/kaif/model_outputs/Deepseek-7b-Instruct/Intermediate_Algebra/predictions2.pkl
Loading output from /lfs/skampere1/0/kaif/model_outputs/Deepseek-7b-Instruct/Number_Theory/predictions2.pkl
Loading output from /lfs/skampere1/0/kaif/model_outputs/Deepseek-7b-Instruct/Prealgebra/predictions2.pkl
Loading output from /lfs/skampere1/0/kaif/model_outputs/Deepseek-7b-Instruct/Precalculus/predictions2.pkl
Loading output from /lfs/skampere1/0/kaif/model_outputs/Deepseek-7b-Instruct/GSM8K/predictions2.pkl


In [13]:
predictions['Algebra'][0]

RequestOutput(request_id=0, prompt='Answer the following 4 questions:\n\n1. Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).\n\nFor the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \\Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\\boxed{0}$.\n\n\n2. A rectangular band formation is a formation with $m$ band members in each of $r$ rows, where $m$ and $r$ are integers. A particular band has less than 100 band members. The director arranges them in a rectangular formation and finds that he has two memb

In [9]:
from dataset import get_boxed_answer
from latex_formater import latex_deformat
import re


def grade(answers: list[str], targets: list[str]) -> list:
    return [a == t for a, t in zip(answers, targets)]


def grade_predictions(outputs, data, dataset):
    if dataset == 'GSM8K':
        boxed_predictions = []
        for o in outputs:
            try:
                boxed_predictions.append(latex_deformat(o[-1]['content']))
                # remove any characters that are not numbers
                boxed_predictions = [re.sub(r'[^0-9]', '', p) for p in boxed_predictions]
            except:
                # print(get_boxed_answer(o.outputs[0].text))
                print(o[-1]['content'])
        # use regex to extract the answer
        boxed_answers = [latex_deformat(re.search(r'\n#### (.*)', d).group(1)) for d in data]
        boxed_answers = [re.sub(r'[^0-9]', '', p) for p in boxed_answers]

        grades = grade(boxed_predictions, boxed_answers)
        return grades
    else:
        boxed_predictions = []
        for o in outputs:
            try:
                # boxed_predictions.append(latex_deformat(get_boxed_answer(o.outputs[0].text)))
                boxed_predictions.append(latex_deformat(get_boxed_answer(o[-1]['content'])))
            except:
                # print(get_boxed_answer(o.outputs[0].text))
                print(o[-1]['content'])
        # boxed_predictions = [latex_deformat(get_boxed_answer(o.outputs[0].text)) for o in outputs]
        boxed_answers = [latex_deformat(get_boxed_answer(d)) for d in data]
        grades = grade(boxed_predictions, boxed_answers)

        return grades

In [ ]:
# from transformers import AutoTokenizer
#
# tokenizer = AutoTokenizer.from_pretrained(model_name)
#
# messages = [
#     {"role": "user", "content": "Hello, how are you?"},
#
#     {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
#
#     {"role": "user", "content": "I'd like to show off how chat templating works!"},
# ]
# tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [10]:
i = 1
dataset_name = 'GSM8K'
# print(predictions[dataset_name][i].prompt)
# print(predictions[dataset_name][i].outputs[0].text)
# print('=' * 50)
# print(datasets_subset[dataset_name][i]['answer'])
print(predictions[dataset_name][i][-4:])
print('=' * 50)
print(datasets_subset[dataset_name][i]['answer'])

[{'role': 'user', 'content': "Patricia has 4 barrels of 7 gallons each. All of them are empty, and there's only one faucet with a flow rate of 3.5 gallons per minute. How many minutes will it take to fill all 4 barrels?"}, {'role': 'assistant', 'content': 'Each barrel is 7 gallons and Patricia has 4 barrels, so the total volume to be filled is 7 gallons/barrel * 4 barrels = 28 gallons.\n\nThe faucet has a flow rate of 3.5 gallons per minute. To find out how long it takes to fill 28 gallons, we divide the total volume by the flow rate:\n\n28 gallons / 3.5 gallons/minute = 8 minutes.\n\nSo, it will take 8 minutes to fill all 4 barrels.'}, {'role': 'user', 'content': 'FINAL ANSWER: '}, {'role': 'assistant', 'content': '8 minutes'}]
How many gallons does Patricia need? ** If Patricia has 4 barrels and each barrel has 7 gallons, then in total she needs 4*7=<<4*7=28>>28 gallons
How many minutes will it take to fill all 4 barrels? ** If the faucet can provide 3.5 gallons every minute, then sh

In [9]:
import numpy as np

grades = {
    dataset: grade_predictions(predictions[dataset], [p['answer'] for p in data], dataset)
    for dataset, data in datasets_subset.items()
}
{dataset: np.mean(grade) for dataset, grade in grades.items()}

{'Algebra': 0.58,
 'Counting_and_Probability': 0.33,
 'Geometry': 0.27,
 'Intermediate_Algebra': 0.24,
 'Number_Theory': 0.42,
 'Prealgebra': 0.61,
 'Precalculus': 0.33,
 'GSM8K': 0.89}

In [10]:
fix_mistake_prompt = [
    ["There is a mistake in this response. Explain the mistake.",
     "Craft a response that doesn't make this mistake. Do not apologize for the mistake or make any similar comment. Just create a new response that doesn't make the mistake that can replace the old response."],
    ["What is wrong with this answer?",
     "Now construct a response that fixes this mistake. Do not apologize for the mistake. Simply answer the question correctly without making any other comments."]
]

fix_mistake_prompts = [
    {
        dataset:
            [
                [
                    {"role": "user", "content": o[-4]['content']},
                    {"role": "assistant", "content": o[-3]['content']},
                ] + [
                    {"role": "user", "content": p}
                    for p in fix_mistake_prompt[cp_idx]
                ]
                for o in predictions[dataset]
            ]
        for dataset, data in datasets_subset.items()
    }
    for cp_idx in range(len(fix_mistake_prompt))
]

In [11]:
# ask how confident the model is in its predictions
confidence_prompt = [
    "How confident are you this is correct? Explain your reasoning. Give a 1-5 rating, 1 being not confident at all and 5 being very confident. End your response with 'Rating: ' followed by your rating.",
    "Are you sure this is correct? Give your confidence as a percentage from 0 to 100.\n\nConfidence: ",
    "Are you sure this is correct? Explain your reasoning and give your confidence as a percentage from 0% to 100%.",
    "How confident are you that this proof is right? Explain your reasoning and say whether you are very confident, confident, somewhat confident, not very confident, or not confident.",
    "Above is a student response to a question. Pretend you are a teacher grading the response. How confident are you that the student's response is correct? Explain your reasoning and give your confidence as a percentage from 0% to 100%.",
    "Above is a student response to a question on a test. You are an expert and nuanced grader who has a great intuition for these things. How confident are you that the student's response is correct? Explain your reasoning and give your confidence as a percentage from 0% to 100%.",
    "Above is a student response to a question on a test. Are there any mistakes in the student's response? If so, explain the mistake and say the student's response is incorrect. If not, say the student's response is correct.",
    "Are there any mistakes in this answer? If so, explain the mistake and say the student's response is incorrect. If not, say the student's response is correct.",
    "Are you sure this is correct? Explain your reasoning and give your confidence as a percentage from 0% to 100%.",
    "How confident are you that there is a mistake in this answer? Explain your reasoning and give your confidence as a percentage from 0% to 100%.",
]

# cp_idx = 1


# concat the confidence prompts to the original prompts and the model's predictions
confidence_prompts = [
    {
        dataset:
            [
                [
                    {"role": "user", "content": o[-4]['content']},
                    {"role": "assistant", "content": o[-3]['content']},
                    {"role": "user", "content": confidence_prompt[cp_idx]}
                ]
                for o in predictions[dataset]
            ]
        for dataset, data in datasets_subset.items()
    }
    for cp_idx in range(len(confidence_prompt))
]

In [69]:
print(fix_mistake_prompts[0]['Algebra'][3])

[{'role': 'user', 'content': 'What is the product of the solutions of the equation $-35=-x^2-2x?$\n\nWrap your final answer with the `\\boxed{}` command.'}, {'role': 'assistant', 'content': "To find the product of the solutions of the quadratic equation $-35 = -x^2 - 2x$, we first need to rearrange the equation and then solve for $x$. \n\nStarting with:\n\\[ -35 = -x^2 - 2x \\]\n\nAdding $x^2$ to both sides to get rid of the negative on the left side:\n\\[ x^2 + 2x - 35 = 0 \\]\n\nNow, we can factor the quadratic or use the quadratic formula. Since this is a quadratic in standard form, it's straightforward to factor:\n\n\\[ (x + 7)(x - 5) = 0 \\]\n\nThe solutions occur when the factors equal zero, so we have:\n\\[ x + 7 = 0 \\quad \\text{or} \\quad x - 5 = 0 \\]\n\nSolving for $x$ gives us:\n\\[ x_1 = -7 \\quad \\text{and} \\quad x_2 = 5 \\]\n\nThe product of the solutions is:\n\\[ \\boxed{x_1 \\cdot x_2 = (-7) \\cdot 5 = -35} \\]"}, {'role': 'user', 'content': "There is a mistake in t

In [11]:
load = True
fix_mistake_predictions = [
    {
        dataset: generate(
            prompts, llm, tokenizer, sampling_params, load=load, override=True,
            output_path=save_dir / MODEL_MAP[model_name] / dataset / f"fix_mistake_{cp_idx}.pkl"
        ) for dataset, prompts in fix_mistake_prompts[cp_idx].items()
    }
    for cp_idx in range(len(fix_mistake_prompt))
]

Processed prompts: 100%|██████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 23.95it/s]


In [66]:
load = True
confidence_predictions = [
    {
        dataset: generate(
            prompts, llm, tokenizer, sampling_params, load=load,
            output_path=save_dir / MODEL_MAP[model_name] / dataset / f"confidence_predictions_{cp_idx}.pkl"
        ) for dataset, prompts in confidence_prompts[cp_idx].items()
    }
    for cp_idx in range(len(confidence_prompt))
]

Loading output from /lfs/skampere1/0/kaif/generated_predictions/Algebra_Qwen-7b_confidence_predictions_0.pkl
Loading output from /lfs/skampere1/0/kaif/generated_predictions/Counting and Probability_Qwen-7b_confidence_predictions_0.pkl
Loading output from /lfs/skampere1/0/kaif/generated_predictions/Geometry_Qwen-7b_confidence_predictions_0.pkl
Loading output from /lfs/skampere1/0/kaif/generated_predictions/Intermediate Algebra_Qwen-7b_confidence_predictions_0.pkl
Loading output from /lfs/skampere1/0/kaif/generated_predictions/Number Theory_Qwen-7b_confidence_predictions_0.pkl
Loading output from /lfs/skampere1/0/kaif/generated_predictions/Prealgebra_Qwen-7b_confidence_predictions_0.pkl
Loading output from /lfs/skampere1/0/kaif/generated_predictions/Precalculus_Qwen-7b_confidence_predictions_0.pkl
Loading output from /lfs/skampere1/0/kaif/generated_predictions/GSM8K_Qwen-7b_confidence_predictions_0.pkl
Loading output from /lfs/skampere1/0/kaif/generated_predictions/Algebra_Qwen-7b_confid

Processed prompts: 100%|██████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  7.94it/s]


In [22]:
grades = {
    dataset: grade_predictions(predictions[dataset], [p['answer'] for p in data], dataset)
    for dataset, data in datasets_subset.items()
}
{dataset: np.mean(grade) for dataset, grade in grades.items()}

{'Algebra': 0.58,
 'Counting_and_Probability': 0.33,
 'Geometry': 0.27,
 'Intermediate_Algebra': 0.24,
 'Number_Theory': 0.42,
 'Prealgebra': 0.61,
 'Precalculus': 0.33,
 'GSM8K': 0.89}

In [12]:
grades = {
    dataset: grade_predictions(fix_mistake_predictions[0][dataset], [p['answer'] for p in data], dataset)
    for dataset, data in datasets_subset.items()
}
{dataset: np.mean(grade) for dataset, grade in grades.items()}

{'Algebra': 0.01,
 'Counting_and_Probability': 0.0,
 'Geometry': 0.0,
 'Intermediate_Algebra': 0.02,
 'Number_Theory': 0.02,
 'Prealgebra': 0.0,
 'Precalculus': 0.0,
 'GSM8K': 0.0}

In [13]:
fix_mistake_predictions[0]['Algebra'][8]

[{'role': 'user',
  'content': 'What is the distance between the center of the circle with equation $x^2+y^2=-4x+6y-12$ and the point $(1,7)$?'},
 {'role': 'assistant',
  'content': ' First, we rewrite the equation of the circle in standard form $(x-a)^2+(y-b)^2=r^2$, where $(a,b)$ is the center of the circle and $r$ is the radius. \n\nTo do this, we complete the square for the $x$ terms and the $y$ terms separately. \n\nThe $x$ terms are $x^2+4x$, and we can complete the square by adding and subtracting $(4/2)^2=4$ to get $(x+2)^2$. \n\nThe $y$ terms are $y^2-6y$, and we can complete the square by adding and subtracting $(6/2)^2=9$ to get $(y-3)^2$. \n\nSo the equation of the circle becomes $(x+2)^2+(y-3)^2=4+9+12=25$. \n\nTherefore, the center of the circle is $(-2,3)$. \n\nThe distance between the center of the circle and the point $(1,7)$ can be found using the distance formula, which is $\\sqrt{(x_2-x_1)^2+(y_2-y_1)^2}$. \n\nSubstituting the coordinates of the center of the circle

In [ ]:
??sns.histplot

In [ ]:
# plot the confidence predictions against grades
import matplotlib.pyplot as plt
import seaborn as sns

num_malformed_confidence_predictions = 0


def get_confidence(confidence_prediction):
    # get the confidence from the confidence prediction
    # if not found return -1
    try:
        confidence_part = confidence_prediction.outputs[0].text.split('\n')[0]
        if '%' in confidence_part:
            return int(confidence_part.split('%')[0])
        else:
            return int(confidence_part)
    except:
        global num_malformed_confidence_predictions
        num_malformed_confidence_predictions += 1
        return -1


def get_rating(confidence_prediction):
    # get the rating from the confidence prediction
    # if not found return -1
    # int(o.outputs[0].text.split('Rating: ')[1])
    try:
        rating_part = confidence_prediction.outputs[0].text.split('Rating: ')[1]
        return int(rating_part[0])
    except:
        global num_malformed_confidence_predictions
        num_malformed_confidence_predictions += 1
        # print(confidence_prediction.outputs[0].text)
        # raise ValueError
        return -1


# one plot for each dataset
for dataset, data in confidence_predictions.items():
    confidences = [get_confidence(o) for o in data]
    # distribution of confidence ratings
    # sns.histplot(confidences)
    # plt.title(f"{dataset} confidence ratings")
    # plt.show()
    # plot the confidence ratings against the grades
    correct = [c for g, c in zip(grades[dataset], confidences) if c != -1 and g]
    incorrect = [c for g, c in zip(grades[dataset], confidences) if c != -1 and not g]
    print(np.unique(correct, return_counts=True))
    print(np.unique(incorrect, return_counts=True))
    print(np.unique(confidences, return_counts=True))
    # plot categorical distribution of rating conditioned on correctness
    # x axis is categorical, y axis is density
    # two histograms should be interleaved, not stacked
    sns.histplot([correct, incorrect], stat='probability', common_norm=False, bins=5, multiple='dodge', legend=True)

    plt.title(f"{dataset} confidence ratings")
    plt.show()


In [ ]:
# save the predictions
# for dataset, outputs in predictions.items():
#     with open(save_dir / f"{dataset.replace(' ', '-')}_{MODEL_MAP[model_name]}_predictions.pkl", "wb") as f:
#         pickle.dump(outputs, f)

In [ ]:
def generate(prompts, model, sampling_params, chat_mode=False):
    outputs = llm.generate(prompts, sampling_params)
    if chat_mode:
        return [prompt + o.outputs[0].text for prompt, o in zip(prompts, outputs)]
    else:
        return [o.outputs[0].text for o in outputs]

In [ ]:
prompts = ["""A board game spinner is divided into three parts labeled $A$, $B$  and $C$.
The probability of the spinner landing on $A$ is $\\frac{1}{3}$ and the
probability of the spinner landing on $B$ is $\\frac{5}{12}$.  What is
the probability of the spinner landing on $C$? Express your answer as a
common fraction. Write your final answer as "\\boxed{...}".

Solution:
"""]
# Solution: The spinner is guaranteed to land on exactly one of the three
# regions, so we know that the sum of the probabilities of it landing in
# each region will be 1. If we let the probability of it landing in region
# $C$ be $x$, we then have the equation $1 = \\frac{5}{12}+\\frac{1}{3}+x$,
# from which we have $x=\\boxed{\\frac{1}{4}}$.

# Above is a question and correct solution. What is a mistake someone might make when solving the problem?

# Mistake:"""]
outputs = generate(prompts, llm, sampling_params, chat_mode=True)
print(outputs[0])

In [ ]:
prompts2 = [o + """

Are you sure this is correct?
""" for o in outputs]
outputs2 = generate(prompts2, llm, sampling_params, chat_mode=True)
print(outputs2[0])

In [ ]:
prompts2 = [o + """

Now construct a response that makes this mistake followed 
by a teacher explaining the mistake made. Begin with "Incorrect Solution:" 
and the teacher corrections with "Teacher Correction:".

Incorrect Solution: """ for o in outputs]
outputs2 = generate(prompts2, llm, sampling_params, chat_mode=True)
print(outputs2[0])